# 성능별 노트북 인기 요소
- '다나와'에서 노트북 검색 후 제품, 가격, 스펙 등의 요소들을 DataFrame으로 읽어들임
- 성능별로 순위를 매기고 실제 인기 순위와 일치하는 정도를 파악
- 시각화도 더불어 진행

In [4]:
# 다나와 노트북 검색 1페이지
url_1page = '''http://search.danawa.com/dsearch.php?query=%EB%85%B8%ED%8A%B8%EB%B6%81&tab=main'''

# 다나와 노트북 검색 2페이지
url_2page = '''http://search.danawa.com/dsearch.php?
query={}&originalQuery={}&volumeType=allvs&
page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods'''

# 다나와 노트북 검색 3페이지
url_3page = '''http://search.danawa.com/dsearch.php?
query{}&originalQuery={}&
previousKeyword={}&volumeType=allvs&
page=3&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods'''

'''http://search.danawa.com/dsearch.php?query={}&volumeType=allvs
    &page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods'''

# 해당 url에서 php의 query 변수와 page 변수를 원하는대로 맞춰주면 된다.

In [1]:
# 다나 홈페이지에서 원하는 제품을 검색하는 함수

def searching(product,page=1):
    try:
        if page==1:
            url = 'http://search.danawa.com/dsearch.php?k1=%EB%85%B8%ED%8A%B8%EB%B6%81&module=goods&act=dispMain'
        else:
            url = '''http://search.danawa.com/dsearch.php?query={}&volumeType=allvs
            &page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods'''.format(product,page)
        return url
    except:
        print("product는 문자열로, page는 숫자로 넣었는지 확인하시오.")

In [2]:
url = searching('노트북',1)

In [4]:
from selenium import webdriver
driver = webdriver.Chrome('C:/Users/hp/Desktop/최영조/chromedriver.exe')
driver.get(url)

In [5]:
html = driver.page_source

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(html,'html.parser')

In [8]:
full_info = soup.select('div.prod_main_info')
first_info = full_info[0]
first_info

<div class="prod_main_info">
<div class="thumb_info">
<div class="thumb_image">
<a class="thumb_link click_log_product_adReader_img_" data-sale-seq="" href="http://prod.danawa.com/info/?pcode=10094031&amp;keyword=%EB%85%B8%ED%8A%B8%EB%B6%81&amp;cate=112758" id="thumbLink_10094031_adLeader" onmousedown="_trkEventLog('통합검색_광고상품_LG전자 2020 그램17 17ZD90N-VX70K(SSD 256GB)');" target="_blank">
<img alt="LG전자 2020 그램17 17ZD90N-VX70K (SSD 256GB)" onerror="this.src='http://img.danawa.com/new/noData/img/noImg_130.gif'" src="http://img.danawa.com/prod_img/500000/031/094/img/10094031_1.jpg?shrink=160:160&amp;_v=20200306134324"/>
</a>
<a class="btn_view_zoom" data-assemblygallerycategory="N" href="#" id="imageZoom_10094031_adLeader" onclick="return false;"><span class="ico i_zoom_vw"></span>이미지보기</a>
</div>
</div>
<div class="prod_info">
<p class="prod_name click_log_product_adReader_title_">
<a href="http://prod.danawa.com/info/?pcode=10094031&amp;keyword=%EB%85%B8%ED%8A%B8%EB%B6%81&amp;cate=112758"

In [9]:
name = first_info.select('p>a')[0].text
name

'LG전자 2020 그램17 17ZD90N-VX70K (SSD 256GB)'

In [10]:
spec = first_info.select('div.spec_list')[0].text.replace("\t","").replace("\n","")
spec

'43.1cm(17인치)  / 인텔  / 코어i7-10세대  / 아이스레이크  / i7-1065G7 1.3GHz(3.9GHz) / 쿼드코어  / 2560x1600(WQXGA)  / 광시야각(IPS)  / 슬림형 베젤  / DDR4  / 8GB  / M.2(NVMe)  / 256GB  / Iris Plus Graphics / 802.11ax 무선랜 / 블루투스  / HDMI  / 웹캠(HD)  / 썬더볼트 : 1개  / USB 3.1 Type-C  / USB 3.1  / UFS  / MicroSD카드 / 숫자 키패드  / 키보드 라이트  / MIL-STD  / 지문 인식  / USB-PD  / 80Wh  / 운영체제 미포함  / 두께 : 17.4mm  / 1.35kg  / 일반유통상품 / 용도 : 사무/인강용 , 휴대용  / 색상: 화이트'

In [11]:
price = int(first_info.select('p.price_sect strong')[0].text.replace(",",""))
price

1642000

In [12]:
# 여러 개의 가격이 표시되는 경우가 종종 있는데 기본 옵션만 포함된 첫 째 가격을 고려하도록 합시다.
full_info[4].select('p.price_sect strong')

[<strong>3,017,000</strong>]

In [13]:
searching('노트북',2)

'http://search.danawa.com/dsearch.php?query=노트북&volumeType=allvs\n            &page=2&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods'

In [14]:
import pandas as pd
import time
from tqdm import tqdm_notebook

In [15]:
# 1페이지 외에 다른 페이지들도 정상적으로 작동하는지 체크해본다.

url = searching('노트북',2)
driver.get(url)
time.sleep(2)
information=[]
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
full_info = soup.select('div.prod_main_info')
time.sleep(1)

for i in tqdm_notebook(range(30)):
    item_info = full_info[i]
    name = item_info.select('p>a')[0].text
    spec = item_info.select('div.spec_list')[0].text.replace("\t","").replace("\n","")
    price = int(item_info.select('p.price_sect strong')[0].text.replace(",",""))
    information.append([name,spec,price])
information

IndexError: list index out of range

In [ ]:
# 전체 페이지를 순환하는 과정이 문제가 없는지 체크해본다.
information=[]
from tqdm import tqdm_notebook
import pandas as pd
import time

product='노트북'
total_page=10

driver = webdriver.Chrome('C:/Users/hp/Desktop/최영조/chromedriver.exe')

for k in tqdm_notebook(range(1,total_page+1)):
    url = searching(product,k)
    driver.get(url)
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    full_info = soup.select('div.prod_main_info')
    time.sleep(1)
    
    if k == 1:
        l=32
    else:
        l=30

    for i in range(l):
        item_info = full_info[i]
        name = item_info.select('p>a')[0].text
        spec = item_info.select('div.spec_list')[0].text.replace("\t","").replace("\n","")
        
        
        # 이미 한번 돌려본 결과 품절된 제품은 가격이 표시되지 않아 에러가 났었음. 그러한 에러를 방지하기 위함
        try:
            price = int(item_info.select('p.price_sect strong')[0].text.replace(",",""))
        except:
            price= None
        information.append([name,spec,price])
        
info2_df = pd.DataFrame(information,colums=['제품명','스펙','가격'])
info2_df.head()

In [18]:
# 정상적으로 작동한다면 이를 함수로 만들어 사용한다.

def get_information(product,total_page):
    information=[]
    from tqdm import tqdm_notebook
    import pandas as pd
    import time


    driver = webdriver.Chrome('C:/Users/hp/Desktop/최영조/chromedriver.exe')

    for k in tqdm_notebook(range(1,total_page+1)):
        url = searching(product,k)
        driver.get(url)
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        full_info = soup.select('div.prod_main_info')
        time.sleep(1)

        if k == 1:
            l=32
        else:
            l=30

        for i in range(l):
            item_info = full_info[i]
            name = item_info.select('p>a')[0].text
            spec = item_info.select('div.spec_list')[0].text.replace("\t","").replace("\n","")
            try:
                price = int(item_info.select('p.price_sect strong')[0].text.replace(",",""))
            except:
                price= None
            information.append([name,spec,price])

    info_df = pd.DataFrame(information,columns=['제품명','스펙','가격'])
    return info_df
                        
                        


In [19]:
result = get_information('노트북',10)

In [20]:
result.head()

,제품명,스펙,가격
0,LG전자 2020 그램17 17ZD90N-VX70K (SSD 256GB),43.1cm(17인치) / 인텔 / 코어i7-10세대 / 아이스레이크 / i...,1692000.0
1,레노버 LEGION Y540-15IRH i7 Prime (SSD 256GB),39.62cm(15.6인치) / 인텔 / 코어i7-9세대 / 커피레이크-R ...,1190000.0
2,MSI 크리에이터 17M A9SD-i7,43.94cm(17.3인치) / 인텔 / 코어i7-9세대 / 커피레이크-R / i7...,1369000.0
3,LG전자 2019 울트라PC 15UD590-GX50K WIN10,39.62cm(15.6인치) / 인텔 / 코어i5-8세대 / 위스키레이크 / i5-...,903000.0
4,LG전자 2020 울트라PC 15U70N-GR56K,39.6cm(15.6인치) / 인텔 / 코어i5-10세대 / 코멧레이크 / i5-1...,1059000.0


In [22]:
result.isnull().sum()
# 가격에 하나의 결측치가 존재

제품명    0
스펙     0
가격     1
dtype: int64

In [21]:
# 정상적으로 나온 결과를 엑셀로 저장함
result.to_excel('notebook.xlsx')

## 크롤링해서 얻은 데이터를 전처리

In [16]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [17]:
result = pd.read_excel('notebook.xlsx')
result.head()

,Unnamed: 0,제품명,스펙,가격
0,0,LG전자 2020 그램17 17ZD90N-VX70K (SSD 256GB),43.1cm(17인치) / 인텔 / 코어i7-10세대 / 아이스레이크 / i...,1692000.0
1,1,레노버 LEGION Y540-15IRH i7 Prime (SSD 256GB),39.62cm(15.6인치) / 인텔 / 코어i7-9세대 / 커피레이크-R ...,1190000.0
2,2,MSI 크리에이터 17M A9SD-i7,43.94cm(17.3인치) / 인텔 / 코어i7-9세대 / 커피레이크-R / i7...,1369000.0
3,3,LG전자 2019 울트라PC 15UD590-GX50K WIN10,39.62cm(15.6인치) / 인텔 / 코어i5-8세대 / 위스키레이크 / i5-...,903000.0
4,4,LG전자 2020 울트라PC 15U70N-GR56K,39.6cm(15.6인치) / 인텔 / 코어i5-10세대 / 코멧레이크 / i5-1...,1059000.0


### 스펙에 해당하는 컬럼이 난잡한 문자열로 되어 있으니 필요한 성능만을 꺼내도록 합시다.
(노트북을 잘 모르기에 성능에 관한 것은 인터넷 정보에 상당부분 의존했습니다.)
- CPU: 셀러론->팬티엄->i3->i5, 세대는 최신으로 올수록 좋은 것(9세대<10세대)

In [9]:
result.스펙[0]

# CPU : 코어i7-10세대
# 무게 : 1.35kg
# SSD : 256GB
# 배터리 : 80kw
# 운영체재 미포함
# 지문인식

'43.1cm(17인치)  / 인텔  / 코어i7-10세대  / 아이스레이크  / i7-1065G7 1.3GHz(3.9GHz) / 쿼드코어  / 2560x1600(WQXGA)  / 광시야각(IPS)  / 슬림형 베젤  / DDR4  / 8GB  / M.2(NVMe)  / 256GB  / Iris Plus Graphics / 802.11ax 무선랜 / 블루투스  / HDMI  / 웹캠(HD)  / 썬더볼트 : 1개  / USB 3.1 Type-C  / USB 3.1  / UFS  / MicroSD카드 / 숫자 키패드  / 키보드 라이트  / MIL-STD  / 지문 인식  / 80Wh  / 운영체제 미포함  / 두께 : 17.4mm  / 1.35kg  / 일반유통상품 / 용도 : 사무/인강용 , 휴대용  / 색상: 화이트'

# 새로 만든 것
# 나중에 수정

In [21]:
result.drop(['Unnamed: 0'],axis=1,inplace=True)

In [22]:
result

,제품명,스펙,가격
0,LG전자 2020 그램17 17ZD90N-VX70K (SSD 256GB),43.1cm(17인치) / 인텔 / 코어i7-10세대 / 아이스레이크 / i...,1692000.0
1,레노버 LEGION Y540-15IRH i7 Prime (SSD 256GB),39.62cm(15.6인치) / 인텔 / 코어i7-9세대 / 커피레이크-R ...,1190000.0
2,MSI 크리에이터 17M A9SD-i7,43.94cm(17.3인치) / 인텔 / 코어i7-9세대 / 커피레이크-R / i7...,1369000.0
3,LG전자 2019 울트라PC 15UD590-GX50K WIN10,39.62cm(15.6인치) / 인텔 / 코어i5-8세대 / 위스키레이크 / i5-...,903000.0
4,LG전자 2020 울트라PC 15U70N-GR56K,39.6cm(15.6인치) / 인텔 / 코어i5-10세대 / 코멧레이크 / i5-1...,1059000.0
...,...,...,...
297,레노버 씽크북 14s-IML 20RSA001KR,35.56cm(14인치) / 인텔 / 코어i7-10세대 / 코멧레이크 / i7-10...,1025000.0
298,한성컴퓨터 TFG277GSW 아방가르드,43.94cm(17.3인치) / 인텔 / 코어i7-9세대 / 커피레이크-R / i7...,1988000.0
299,레노버 LEGION Y540-15IRH i7 Blade 2060,39.62cm(15.6인치) / 인텔 / 코어i7-9세대 / 커피레이크-R / i7...,1397000.0
300,MSI WT73VR 7RM,43.94cm(17.3인치) / 인텔 / 제온 / 카비레이크 / E3-1505Mv6...,6490000.0


In [23]:
result.isnull().sum()

제품명    0
스펙     0
가격     1
dtype: int64

In [27]:
result2 = result[result['가격'].isnull()==False]

In [29]:
result2.isnull().sum()

제품명    0
스펙     0
가격     0
dtype: int64